<a href="https://colab.research.google.com/github/thravt/FederatedAggregation/blob/main/CNNLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def average_weights(models):
    weights = [model.get_weights() for model in models]
    new_weights = list()
    for weights_list_tuple in zip(*weights):
        new_weights.append(np.array(weights_list_tuple).mean(axis=0))
    return new_weights

In [ ]:
def makecnn():
  model = tf.keras.models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))
  return model

In [ ]:
import matplotlib.pyplot as plt
import PIL
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


For this I will use 5 local models, and a central model.

In [ ]:
modelC = makecnn()
modellist = []
for i in range(5):
  modellist.append(makecnn())
  modellist[i].set_weights(list(modelC.get_weights()))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Getting splits:
imagelist = [[], [], [], [], []]
labellist = [[], [], [], [], []]
for i in range (len(train_labels)):
  imagelist[i % 5].append(train_images[i])
  labellist[i % 5].append(train_labels[i])


In [ ]:
for i in range(5):
  imagelist[i] = np.stack(imagelist[i])
  labellist[i] = np.stack(labellist[i])

In [ ]:
for i in range(5):
  modellist[i].compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

In [ ]:
modelC.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

In [ ]:
def layer_discrepancy(layer):
  discrepancysum = 0
  for i in range(5):
    discrepancysum = discrepancysum + tf.norm((modellist[i].get_weights()[layer] - modelC.get_weights()[layer]), ord='euclidean')**2
  discrepancysum = discrepancysum / 5
  return discrepancysum

Testing

In [ ]:
print(layer_discrepancy(0))

tf.Tensor(0.0, shape=(), dtype=float32)


Layerwise adjustment check

In [ ]:
def get_intervals(discrepancyarray, increasing_factor, base_interval):
  totaldiscrepancy = np.sum(discrepancyarray)
  total_params = sum(layer.count_params() for layer in modelC.layers)
  intervals = []
  for l in range (8):
    partialdiscrepancy = 0
    for i in range(l + 1):
      partialdiscrepancy = partialdiscrepancy + discrepancyarray[i]
    partialdiscrepancy = partialdiscrepancy / totaldiscrepancy
    partialparams = 0
    for i in range(l + 1):
      partialparams = partialparams + modelC.layers[i].count_params()
    partialparams = partialparams / total_params
    if(partialdiscrepancy < partialparams):
      intervals.append(increasing_factor * base_interval)
    else:
      intervals.append(base_interval)
  return intervals



In [ ]:
for i in range (8):
  print(layer_discrepancy(i))

tf.Tensor(0.09407562, shape=(), dtype=float32)
tf.Tensor(0.00955695, shape=(), dtype=float32)
tf.Tensor(8.6733675, shape=(), dtype=float32)
tf.Tensor(0.032126162, shape=(), dtype=float32)
tf.Tensor(22.353968, shape=(), dtype=float32)
tf.Tensor(0.0197856, shape=(), dtype=float32)
tf.Tensor(62.043617, shape=(), dtype=float32)
tf.Tensor(0.023917187, shape=(), dtype=float32)


In [ ]:
discreps = []
for i in range(8):
  discreps.append(layer_discrepancy(i))

In [ ]:
print(get_intervals(discreps, 2, 5))

[10, 10, 10, 10, 10, 10, 5, 5]


In [ ]:
def average_weights(models, layer):
    weights = [model.layers[layer].get_weights() for model in models]
    new_weights = list()
    for weights_list_tuple in zip(*weights):
        new_weights.append(np.array(weights_list_tuple).mean(axis=0))
    return new_weights

In [ ]:
layerupdatecounter = [0, 0, 0, 0, 0, 0, 0, 0]
intervals = [3, 3, 3, 3, 3, 3, 3, 3]
for i in range(25):
  for j in range(5):
    modellist[j].fit(imagelist[j], labellist[j], epochs = 1)


  discrepancyarray = []
  for j in range(8):
    discrepancyarray.append(layer_discrepancy(j))
  intervals = get_intervals(discrepancyarray, 2, 3)
  print(intervals)
  for k in range(8):
    if (layerupdatecounter[k] >= intervals[k]):
      averageweights = average_weights(modellist, k)
      modelC.layers[k].set_weights(averageweights)
      for j in range(5):
        modellist[j].layers[k].set_weights(list(modelC.layers[k].get_weights()))
      layerupdatecounter[k] = 1
    else:
      layerupdatecounter[k] = layerupdatecounter[k] + 1

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.1305 - f1_score: 0.1922 - loss: 4.4158
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.1454 - f1_score: 0.1943 - loss: 4.9924
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.1410 - f1_score: 0.1930 - loss: 4.4929
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.1483 - f1_score: 0.1945 - loss: 4.5997
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.1301 - f1_score: 0.1876 - loss: 4.3790
[3, 3, 3, 3, 3, 3, 3, 3]
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.2394 - f1_score: 0.1917 - loss: 2.0698
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.2934 - f1_score: 0.1947 - loss: 1.9323
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.2574 - f1_score: 0.1926 - loss: 1.9261
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.2800 - f1_score: 0.1951 - loss: 1.9627
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.2016 - f1_score: 0.1892 - loss: 2.0982
[6, 3, 3, 3, 3,

In [ ]:
modelC.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6886 - f1_score: 0.1955 - loss: 1.0266


[1.033232569694519,
 0.6776000261306763,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.05093571, 0.09718993, 0.12133208, 0.1564725 , 0.18819445,
        0.1729138 , 0.2636425 , 0.27325466, 0.31985947, 0.31182396],
       dtype=float32)>]